In [2]:
# Turing Machine to calculate Fibonacci Sequence
# input = n
# output = F(n)

In [78]:
def parse(defn):
    states = {}
    for s in defn:
        state_name = s.split(":")[0]
        t_string = s.split(":")[1].strip().split(" ")
#         print(t_string)
        assert (len(t_string) == 3)
        transition = {}
        for t in t_string:
            read = t[0]
            write = t[1]
            direction = t[2]
            next_state = t[3:]
            transition[read] = {"write": write, "direction": direction, "next":next_state}
        states[state_name] = transition
    return states

def run(machine, tape, debug=False):
    state = "0"
    head = 0
    while state != "HALT":
        if debug == True:
            print_tape(tape, head)
            
        head_char = tape[head]
        write = machine[state][head_char]["write"]
        print("state ", state, "read", head_char, "write" , write)
        tape[head] = write
        direction = machine[state][head_char]["direction"]
        next_state = machine[state][head_char]["next"]
        
        if direction == "R":
            head += 1
            if head >= len(tape):
                tape.append("_")
        else:
            head -= 1
            if head < 0:
                tape.insert(0,"_")
                head += 1
        
        state = next_state
        
    if debug == True:
        print_tape(tape, head)

def print_tape(tape, head):
    for i in range(0,head):
        print(tape[i], end="")
    print("^", end="")
    for i in range(head, len(tape)):
        print(tape[i], end="")
    print()
    

In [101]:
#Format: State number: (input<0,1,_>,output<0,1,_>,head<L,R>,newstate) x 3
#Machine: reads input, writes output to tape, moves in the direction specified
#Go to the end of the input
machine = [
    "0: 00R0 11R0 __Rinit0",
    #write 00_01_ (the last blank is implicit)
    "init0: 00Rinit1 10Rinit1 _0Rinit1",
    "init1: 00Rinit2 10Rinit2 _0Rinit2",
    "init2: 0_Rinit3 1_Rinit3 __Rinit3",
    "init3: 00Rinit4 10Rinit4 _0Rinit4",
    "init4: 01Rinit5 11Rinit5 _1Rinit5",
    "init5: 0_Lback0 1_Lback0 __Lback0", #moving L because we wrote a 1 already, so now we want to go back
    # tape now is "n_0*_1", go back to n
    "back0: 00Lback0 11Lback0 __Lback1",
    "back1: 00Lback1 11Lback1 __Lback2",
    "back2: 00Lback2 11Lback2 __Rcheckn0", #moving R because we were at a blank (to the left of n)
    #back at beginning, check to see if n == 0
    "checkn0: 00Rcheckn0 11Rsub0 __Rclean",
    # state 7 means n != 0
    # move to the right of n
    "sub0: 00Rsub0 11Rsub0 __Lsub1",
    # now at least significant bit of n, subtract 1
    "sub1: 01Lsub1 10Lsub2 __RHALT", # should never reach _
    "sub2: 00Lsub2 11Lsub2 __Rfib0",
    # state clean means n == 0  
    "clean: 00LHALT 11LHALT __LHALT",
    # state fib0 means we have to start adding the stuff (we are at leftmost bit of n)
    "fib0: 00Rfib0 11Rfib0 __Rfib1",
    "fib1: 00Rfib1 11Rfib1 __Rfib2",
    "fib2: 00Rfib2 11Rfib2 __Rfib3",
    "fib3: 00Rfib3 11Rfib3 _1Lfib4", # for now just keep on adding 1's to the end of the list (eventually do the sum)
    "fib4: 00Lfib4 11Lfib4 __Lback0", # we need to go back one extra "_" before going to the back subroutine
]

In [107]:
mach = parse(machine)
run(mach, tape = ["1", "1", "1", "0", "1"], debug=True)

^11101
state  0 read 1 write 1
1^1101
state  0 read 1 write 1
11^101
state  0 read 1 write 1
111^01
state  0 read 0 write 0
1110^1
state  0 read 1 write 1
11101^_
state  0 read _ write _
11101_^_
state  init0 read _ write 0
11101_0^_
state  init1 read _ write 0
11101_00^_
state  init2 read _ write _
11101_00_^_
state  init3 read _ write 0
11101_00_0^_
state  init4 read _ write 1
11101_00_01^_
state  init5 read _ write _
11101_00_0^1_
state  back0 read 1 write 1
11101_00_^01_
state  back0 read 0 write 0
11101_00^_01_
state  back0 read _ write _
11101_0^0_01_
state  back1 read 0 write 0
11101_^00_01_
state  back1 read 0 write 0
11101^_00_01_
state  back1 read _ write _
1110^1_00_01_
state  back2 read 1 write 1
111^01_00_01_
state  back2 read 0 write 0
11^101_00_01_
state  back2 read 1 write 1
1^1101_00_01_
state  back2 read 1 write 1
^11101_00_01_
state  back2 read 1 write 1
^_11101_00_01_
state  back2 read _ write _
_^11101_00_01_
state  checkn0 read 1 write 1
_1^1101_00_01_
state  sub0

_1^0111_00_01_111111
state  back2 read 0 write 0
_^10111_00_01_111111
state  back2 read 1 write 1
^_10111_00_01_111111
state  back2 read _ write _
_^10111_00_01_111111
state  checkn0 read 1 write 1
_1^0111_00_01_111111
state  sub0 read 0 write 0
_10^111_00_01_111111
state  sub0 read 1 write 1
_101^11_00_01_111111
state  sub0 read 1 write 1
_1011^1_00_01_111111
state  sub0 read 1 write 1
_10111^_00_01_111111
state  sub0 read _ write _
_1011^1_00_01_111111
state  sub1 read 1 write 0
_101^10_00_01_111111
state  sub2 read 1 write 1
_10^110_00_01_111111
state  sub2 read 1 write 1
_1^0110_00_01_111111
state  sub2 read 0 write 0
_^10110_00_01_111111
state  sub2 read 1 write 1
^_10110_00_01_111111
state  sub2 read _ write _
_^10110_00_01_111111
state  fib0 read 1 write 1
_1^0110_00_01_111111
state  fib0 read 0 write 0
_10^110_00_01_111111
state  fib0 read 1 write 1
_101^10_00_01_111111
state  fib0 read 1 write 1
_1011^0_00_01_111111
state  fib0 read 0 write 0
_10110^_00_01_111111
state  fib0 r

_1001^0_00_01_1111111111
state  fib0 read 0 write 0
_10010^_00_01_1111111111
state  fib0 read _ write _
_10010_^00_01_1111111111
state  fib1 read 0 write 0
_10010_0^0_01_1111111111
state  fib1 read 0 write 0
_10010_00^_01_1111111111
state  fib1 read _ write _
_10010_00_^01_1111111111
state  fib2 read 0 write 0
_10010_00_0^1_1111111111
state  fib2 read 1 write 1
_10010_00_01^_1111111111
state  fib2 read _ write _
_10010_00_01_^1111111111
state  fib3 read 1 write 1
_10010_00_01_1^111111111
state  fib3 read 1 write 1
_10010_00_01_11^11111111
state  fib3 read 1 write 1
_10010_00_01_111^1111111
state  fib3 read 1 write 1
_10010_00_01_1111^111111
state  fib3 read 1 write 1
_10010_00_01_11111^11111
state  fib3 read 1 write 1
_10010_00_01_111111^1111
state  fib3 read 1 write 1
_10010_00_01_1111111^111
state  fib3 read 1 write 1
_10010_00_01_11111111^11
state  fib3 read 1 write 1
_10010_00_01_111111111^1
state  fib3 read 1 write 1
_10010_00_01_1111111111^_
state  fib3 read _ write 1
_10010_00_0

_01111_00_01_11111111111^11
state  fib3 read 1 write 1
_01111_00_01_111111111111^1
state  fib3 read 1 write 1
_01111_00_01_1111111111111^_
state  fib3 read _ write 1
_01111_00_01_111111111111^11
state  fib4 read 1 write 1
_01111_00_01_11111111111^111
state  fib4 read 1 write 1
_01111_00_01_1111111111^1111
state  fib4 read 1 write 1
_01111_00_01_111111111^11111
state  fib4 read 1 write 1
_01111_00_01_11111111^111111
state  fib4 read 1 write 1
_01111_00_01_1111111^1111111
state  fib4 read 1 write 1
_01111_00_01_111111^11111111
state  fib4 read 1 write 1
_01111_00_01_11111^111111111
state  fib4 read 1 write 1
_01111_00_01_1111^1111111111
state  fib4 read 1 write 1
_01111_00_01_111^11111111111
state  fib4 read 1 write 1
_01111_00_01_11^111111111111
state  fib4 read 1 write 1
_01111_00_01_1^1111111111111
state  fib4 read 1 write 1
_01111_00_01_^11111111111111
state  fib4 read 1 write 1
_01111_00_01^_11111111111111
state  fib4 read _ write _
_01111_00_0^1_11111111111111
state  back0 read 1 w

_01100_00_0^1_1111111111111111
state  fib2 read 1 write 1
_01100_00_01^_1111111111111111
state  fib2 read _ write _
_01100_00_01_^1111111111111111
state  fib3 read 1 write 1
_01100_00_01_1^111111111111111
state  fib3 read 1 write 1
_01100_00_01_11^11111111111111
state  fib3 read 1 write 1
_01100_00_01_111^1111111111111
state  fib3 read 1 write 1
_01100_00_01_1111^111111111111
state  fib3 read 1 write 1
_01100_00_01_11111^11111111111
state  fib3 read 1 write 1
_01100_00_01_111111^1111111111
state  fib3 read 1 write 1
_01100_00_01_1111111^111111111
state  fib3 read 1 write 1
_01100_00_01_11111111^11111111
state  fib3 read 1 write 1
_01100_00_01_111111111^1111111
state  fib3 read 1 write 1
_01100_00_01_1111111111^111111
state  fib3 read 1 write 1
_01100_00_01_11111111111^11111
state  fib3 read 1 write 1
_01100_00_01_111111111111^1111
state  fib3 read 1 write 1
_01100_00_01_1111111111111^111
state  fib3 read 1 write 1
_01100_00_01_11111111111111^11
state  fib3 read 1 write 1
_01100_00_01_1

...

_00111_00_01_111111^111111111111111
state  fib3 read 1 write 1
_00111_00_01_1111111^11111111111111
state  fib3 read 1 write 1
_00111_00_01_11111111^1111111111111
state  fib3 read 1 write 1
_00111_00_01_111111111^111111111111
state  fib3 read 1 write 1
_00111_00_01_1111111111^11111111111
state  fib3 read 1 write 1
_00111_00_01_11111111111^1111111111
state  fib3 read 1 write 1
_00111_00_01_111111111111^111111111
state  fib3 read 1 write 1
_00111_00_01_1111111111111^11111111
state  fib3 read 1 write 1
_00111_00_01_11111111111111^1111111
state  fib3 read 1 write 1
_00111_00_01_111111111111111^111111
state  fib3 read 1 write 1
_00111_00_01_1111111111111111^11111
state  fib3 read 1 write 1
_00111_00_01_11111111111111111^1111
state  fib3 read 1 write 1
_00111_00_01_111111111111111111^111
state  fib3 read 1 write 1
_00111_00_01_1111111111111111111^11
state  fib3 read 1 write 1
_00111_00_01_11111111111111111111^1
state  fib3 read 1 write 1
_00111_00_01_111111111111111111111^_
state  fib3 read _

_^00101_00_01_11111111111111111111111
state  sub2 read 0 write 0
^_00101_00_01_11111111111111111111111
state  sub2 read _ write _
_^00101_00_01_11111111111111111111111
state  fib0 read 0 write 0
_0^0101_00_01_11111111111111111111111
state  fib0 read 0 write 0
_00^101_00_01_11111111111111111111111
state  fib0 read 1 write 1
_001^01_00_01_11111111111111111111111
state  fib0 read 0 write 0
_0010^1_00_01_11111111111111111111111
state  fib0 read 1 write 1
_00101^_00_01_11111111111111111111111
state  fib0 read _ write _
_00101_^00_01_11111111111111111111111
state  fib1 read 0 write 0
_00101_0^0_01_11111111111111111111111
state  fib1 read 0 write 0
_00101_00^_01_11111111111111111111111
state  fib1 read _ write _
_00101_00_^01_11111111111111111111111
state  fib2 read 0 write 0
_00101_00_0^1_11111111111111111111111
state  fib2 read 1 write 1
_00101_00_01^_11111111111111111111111
state  fib2 read _ write _
_00101_00_01_^11111111111111111111111
state  fib3 read 1 write 1
_00101_00_01_1^1111111111

_00100_00_01^_1111111111111111111111111
state  fib4 read _ write _
_00100_00_0^1_1111111111111111111111111
state  back0 read 1 write 1
_00100_00_^01_1111111111111111111111111
state  back0 read 0 write 0
_00100_00^_01_1111111111111111111111111
state  back0 read _ write _
_00100_0^0_01_1111111111111111111111111
state  back1 read 0 write 0
_00100_^00_01_1111111111111111111111111
state  back1 read 0 write 0
_00100^_00_01_1111111111111111111111111
state  back1 read _ write _
_0010^0_00_01_1111111111111111111111111
state  back2 read 0 write 0
_001^00_00_01_1111111111111111111111111
state  back2 read 0 write 0
_00^100_00_01_1111111111111111111111111
state  back2 read 1 write 1
_0^0100_00_01_1111111111111111111111111
state  back2 read 0 write 0
_^00100_00_01_1111111111111111111111111
state  back2 read 0 write 0
^_00100_00_01_1111111111111111111111111
state  back2 read _ write _
_^00100_00_01_1111111111111111111111111
state  checkn0 read 0 write 0
_0^0100_00_01_1111111111111111111111111
state  

_00010_00_01_1111111111111111111111111^1
state  fib3 read 1 write 1
_00010_00_01_11111111111111111111111111^_
state  fib3 read _ write 1
_00010_00_01_1111111111111111111111111^11
state  fib4 read 1 write 1
_00010_00_01_111111111111111111111111^111
state  fib4 read 1 write 1
_00010_00_01_11111111111111111111111^1111
state  fib4 read 1 write 1
_00010_00_01_1111111111111111111111^11111
state  fib4 read 1 write 1
_00010_00_01_111111111111111111111^111111
state  fib4 read 1 write 1
_00010_00_01_11111111111111111111^1111111
state  fib4 read 1 write 1
_00010_00_01_1111111111111111111^11111111
state  fib4 read 1 write 1
_00010_00_01_111111111111111111^111111111
state  fib4 read 1 write 1
_00010_00_01_11111111111111111^1111111111
state  fib4 read 1 write 1
_00010_00_01_1111111111111111^11111111111
state  fib4 read 1 write 1
_00010_00_01_111111111111111^111111111111
state  fib4 read 1 write 1
_00010_00_01_11111111111111^1111111111111
state  fib4 read 1 write 1
_00010_00_01_1111111111111^11111111

_0000^1_00_01_1111111111111111111111111111
state  sub1 read 1 write 0
_000^00_00_01_1111111111111111111111111111
state  sub2 read 0 write 0
_00^000_00_01_1111111111111111111111111111
state  sub2 read 0 write 0
_0^0000_00_01_1111111111111111111111111111
state  sub2 read 0 write 0
_^00000_00_01_1111111111111111111111111111
state  sub2 read 0 write 0
^_00000_00_01_1111111111111111111111111111
state  sub2 read _ write _
_^00000_00_01_1111111111111111111111111111
state  fib0 read 0 write 0
_0^0000_00_01_1111111111111111111111111111
state  fib0 read 0 write 0
_00^000_00_01_1111111111111111111111111111
state  fib0 read 0 write 0
_000^00_00_01_1111111111111111111111111111
state  fib0 read 0 write 0
_0000^0_00_01_1111111111111111111111111111
state  fib0 read 0 write 0
_00000^_00_01_1111111111111111111111111111
state  fib0 read _ write _
_00000_^00_01_1111111111111111111111111111
state  fib1 read 0 write 0
_00000_0^0_01_1111111111111111111111111111
state  fib1 read 0 write 0
_00000_00^_01_111111

In [108]:
len("11111111111111111111111111111")

29